<a href="https://colab.research.google.com/github/shreekrishh/Data-Analytics/blob/main/sentiment_analysis_on_movie_reviews_through_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelBinarizer
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# read the data
df = pd.read_csv("/content/drive/MyDrive/sentiment analysis on movie review using NLP/files/data/labelled_movie_reviews.csv")



# shuffle the rows
df = df.sample(frac=1, random_state=123).reset_index(drop=True)


In [ ]:

train_frac, val_frac, test_frac = 0.7, 0.1, 0.2
Xr = df["text"].tolist()
Yr = df["label"].tolist()
train_end = int(train_frac*len(Xr))
val_end = int((train_frac + val_frac)*len(Xr))


data = dict(np.load("/content/drive/MyDrive/sentiment analysis on movie review using NLP/files/data/word_vectors.npz",))


w2v = {w:v for w, v in zip(data["words"], data["vectors"])}
print(data['vectors'].shape)




(82197, 300)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
#stopwords = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
unk = np.random.randn(*data["vectors"][0].shape)
unk.shape

(300,)

In [ ]:
word_to_vec_dic = dict(zip(list(data["words"]), list(data["vectors"])))

In [ ]:
# convert a document into a vector
def document_to_vector(doc):
  doc = re.sub("\'", "", doc)
  doc = re.sub("[^a-zA-Z]"," ",doc)
  doc = ' '.join(doc.split())
  doc = doc.lower()
  doc_tok= word_tokenize(doc)
  # doc_tok = tok(doc)
  filtered_words = list(set(doc_tok)-set(stopwords.words('english')))

  for i in range(len(filtered_words)):
    if filtered_words[i] in word_to_vec_dic:
      filtered_words[i] = word_to_vec_dic.get(filtered_words[i])
    else:
      filtered_words[i] = unk
  vec = np.mean(np.array(filtered_words), axis=0)
  return vec

df['vector of text'] = df['text'].apply(lambda x: document_to_vector(x))


In [ ]:

from sklearn.model_selection import train_test_split
train,test = train_test_split(df, test_size=0.2)
train,val = train_test_split(train, train_size = 0.875)
X_train, Y_train = train['vector of text'], train['label']
X_val, Y_val = val['vector of text'], val['label']
X_test, Y_test = test['vector of text'],test['label']


In [ ]:
X_train = np.stack(X_train.values)
Y_train = np.stack(Y_train.values)
X_val = np.stack(X_val.values)
Y_val = np.stack(Y_val.values)
X_test = np.stack(X_test.values)
Y_test = np.stack(Y_test.values)



In [ ]:
Y_train = (Y_train == 'pos').astype(int)
Y_val = (Y_val== 'pos').astype(int)
Y_test = (Y_test == 'pos').astype(int)


In [ ]:

C_set = [0.01,0.05,0.1, 0.3, 0.6, 1, 5,6,7, 10, 20, 25]
for i in C_set:
  lr=LogisticRegression(penalty='l2',max_iter=500,C=i,random_state=42) 
  def fit_model(Xtr, Ytr):
    model=lr.fit(Xtr, Ytr)
    return model
  def test_model(model, Xtst, Ytst):
    return model.score(Xtst, Ytst)
  print(test_model(fit_model(X_train,Y_train), X_val, Y_val))

    

0.813
0.8344
0.8402
0.8468
0.8498
0.8496
0.853
0.8532
0.8536
0.8544
0.8538
0.854
0.853


In [ ]:
#for C = 10, accuracy is maximum 
lr=LogisticRegression(penalty='l2',max_iter=500,C=10,random_state=42)
test_model(fit_model(np.concatenate((X_train,X_val)),np.concatenate((Y_train,Y_val))),X_test,Y_test )

0.8562